In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OpenAI_API_KEY

In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

필수 라이브러리 설치

In [8]:
!pip install numpy pandas --upgrade -q
!pip install langchain langchain-openai datasets openai tiktoken faiss-cpu chromadb gradio==3.50.2 python-dotenv langchain-community langchain-huggingface sentence-transformers -q
print("필수 라이브러리 설치 완료 (또는 이미 설치됨).")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 96.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 3.50.2 requires numpy~=1.0, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [9]:
import os
import pandas as pd
from datasets import load_dataset
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from google.colab import userdata

In [12]:
try:
    dataset = load_dataset("aiqwe/FinShibainu", "qa", split='train[:1000]') # 예시로 1000개만 로드
    df = dataset.to_pandas()
    print(f"데이터셋 로드 완료. 총 {len(df)}개의 샘플.")
    # print(df.head()) # 데이터 구조 확인
except Exception as e:
    print(f"데이터셋 로드 중 오류 발생: {e}")
    df = pd.DataFrame() # 오류 시 빈 DataFrame 생성

데이터셋 로드 완료. 총 1000개의 샘플.


In [13]:
# temperature=0으로 설정하여 일관성 있는 결과 유도
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
print("LLM 초기화 완료 (gpt-4o)")

LLM 초기화 완료 (gpt-4o)


In [14]:
# --- 모델 1: 답변 품질 평가 및 개선 제안 자동화 모델 ---
def evaluate_answer_quality(question: str, answer: str, llm: ChatOpenAI, criteria_examples: str = "") -> str:
    """
    주어진 질문과 답변 쌍의 품질을 평가하고 개선점을 제안하는 함수.
    """
    prompt_text = """
    당신은 금융 챗봇 답변 품질 평가 전문가입니다. 주어진 질문과 답변을 다음 기준에 따라 평가하고, 개선이 필요하다면 구체적인 개선 방안과 예시 답변을 제안해주세요.

    [평가 기준]
    1. 명확성: 답변 내용이 이해하기 쉽고 명료한가? 전문 용어 사용이 적절한가?
    2. 정확성: 답변 내용이 사실에 부합하고 오류가 없는가?
    3. 완결성: 질문의 의도를 충분히 파악하고 관련 정보를 빠짐없이 제공하는가?
    {criteria_section}

    [질문]
    {question}

    [평가 대상 답변]
    {answer}

    [평가 결과 및 개선 제안]
    명확성 점수 (1-5):
    정확성 점수 (1-5):
    완결성 점수 (1-5):
    종합 평가:
    개선 제안 (필요시 구체적인 문구 포함):
    """
    criteria_section = f"\n[참고: 좋은 답변의 특징]\n{criteria_examples}" if criteria_examples else ""
    prompt = PromptTemplate(input_variables=["question", "answer", "criteria_section"], template=prompt_text)
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.invoke({"question": question, "answer": answer, "criteria_section": criteria_section})
    return response['text']

In [15]:
# 모델 1 실행 예시
print("\n--- 모델 1: 답변 품질 평가 실행 예시 ---")
if not df.empty:
    sample_index = 0
    sample_question = df.loc[sample_index, 'question']
    sample_answer_to_evaluate = df.loc[sample_index, 'answer_A']
    good_answer_criteria = df.loc[sample_index, 'preference_desc']
    print(f"질문: {sample_question}")
    print(f"평가 대상 답변: {sample_answer_to_evaluate}")
    print(f"참고 (좋은 답변 기준): {good_answer_criteria}")
    print("\n[LLM 평가 및 제안 결과]")
    evaluation_result = evaluate_answer_quality(sample_question, sample_answer_to_evaluate, llm, good_answer_criteria)
    print(evaluation_result)
else:
    print("데이터프레임이 비어 있어 실행할 수 없습니다.")


--- 모델 1: 답변 품질 평가 실행 예시 ---
질문: 가계부문의 순저축률을 측정하는 공식을 적고, 각 항목이 의미하는 바를 설명하시오.
평가 대상 답변: 가계부문의 순저축률 = (가계부문 순저축액) / (가계 순처분가능소득 + 사회적 현물이전 수취 + 연금기금의 가계 순지분 증감조정액)입니다. 여기서 가계부문 순저축액은 소득에서 소비를 뺀 나머지로, 가계 순처분가능소득은 가계가 실제로 사용할 수 있는 소득을 의미합니다. 사회적 현물이전은 정부나 기부단체에서 제공하는 서비스나 재화이고, 연금기금의 가계 순지분 증감조정액은 가계의 연금부담금과 수취액을 반영하는 항목입니다.
참고 (좋은 답변 기준): answer B는 공식과 각 항목의 의미를 명확히 설명하고 있으며, 순저축률의 중요성과 경제적 함의까지 다루어 경제지표로서의 깊이를 제공하고 있습니다. 반면 answer A는 공식은 제시하였지만, 설명이 다소 간략하고 추가적인 정보 제공이 부족합니다.

[LLM 평가 및 제안 결과]


<ipython-input-14-cc8d774960b2>:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


명확성 점수 (1-5): 4
- 답변은 전반적으로 명확하고 이해하기 쉬운 편입니다. 다만, "연금기금의 가계 순지분 증감조정액"과 같은 용어는 일반 대중에게 다소 어려울 수 있으므로 추가 설명이 필요합니다.

정확성 점수 (1-5): 5
- 답변은 정확하며, 공식과 각 항목의 의미를 올바르게 설명하고 있습니다.

완결성 점수 (1-5): 3
- 질문의 의도를 충분히 파악하고 있지만, 순저축률의 경제적 중요성이나 함의에 대한 설명이 부족합니다. 이러한 부분을 보완하면 더 완성도 높은 답변이 될 것입니다.

종합 평가:
- 답변은 전반적으로 명확하고 정확하지만, 경제적 함의에 대한 설명이 부족하여 완결성이 다소 떨어집니다. 추가적인 설명을 통해 독자가 순저축률의 중요성을 이해할 수 있도록 보완이 필요합니다.

개선 제안:
- "연금기금의 가계 순지분 증감조정액"에 대한 추가 설명을 제공하여 이해를 돕고, 순저축률이 경제 지표로서 가지는 중요성을 설명하는 것이 좋습니다.

예시 답변:
"가계부문의 순저축률 = (가계부문 순저축액) / (가계 순처분가능소득 + 사회적 현물이전 수취 + 연금기금의 가계 순지분 증감조정액)입니다. 여기서 가계부문 순저축액은 소득에서 소비를 뺀 나머지로, 가계가 실제로 저축한 금액을 의미합니다. 가계 순처분가능소득은 세금과 같은 의무 지출을 제외하고 가계가 실제로 사용할 수 있는 소득을 뜻합니다. 사회적 현물이전은 정부나 기부단체에서 제공하는 서비스나 재화로, 가계의 실질적인 소득에 포함됩니다. 연금기금의 가계 순지분 증감조정액은 가계가 연금에 납부한 금액과 수령한 금액의 차이를 반영하여, 가계의 장기적인 재정 상태를 고려한 항목입니다. 순저축률은 가계의 저축 성향을 나타내며, 경제의 안정성과 성장 가능성을 평가하는 중요한 지표로 사용됩니다. 높은 순저축률은 가계가 미래를 대비해 저축을 많이 하고 있음을 의미하며, 이는 경제의 장기적인 투자 여력을 증가시킬 수 있습니다."


In [16]:
# --- 모델 2: 유사 질문 생성 및 발화 패턴 확장 모델 ---
def generate_similar_questions(original_question: str, num_variations: int, llm: ChatOpenAI) -> str:
    """
    원본 질문과 의미는 유사하지만 표현이 다른 질문들을 생성하는 함수.
    """
    prompt_text = """
    당신은 다양한 사용자 발화 패턴을 생성하는 AI입니다. 다음 주어진 금융 관련 질문과 의미는 동일하게 유지하면서, 다른 표현 방식이나 단어를 사용하여 사용자들이 실제로 할 법한 질문 {num_variations}가지를 생성해주세요.

    [생성 시 고려사항]
    - 구어체 표현 포함
    - 간단한 오타나 비문 포함 가능 (선택적)
    - 핵심 키워드는 유지하되 동의어 활용
    - 질문 형식 다양화 (예: ~란?, ~알려줘, ~뭐야?)

    [원본 질문]
    {question}

    [생성된 유사 질문 목록 (번호 매겨서)]
    1.
    2.
    ...
    """
    prompt = PromptTemplate(input_variables=["question", "num_variations"], template=prompt_text)
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.invoke({"question": original_question, "num_variations": num_variations})
    return response['text']

In [17]:
# 모델 2 실행 예시
print("\n--- 모델 2: 유사 질문 생성 실행 예시 ---")
if not df.empty and len(df) > 1:
    sample_index_gen = 1
    sample_question_for_gen = df.loc[sample_index_gen, 'question']
    num_questions_to_generate = 5
    print(f"원본 질문: {sample_question_for_gen}")
    print(f"생성 목표 개수: {num_questions_to_generate}")
    print("\n[LLM 생성 결과]")
    generated_questions = generate_similar_questions(sample_question_for_gen, num_questions_to_generate, llm)
    print(generated_questions)
else:
    print("데이터프레임이 비어 있거나 샘플 수가 부족하여 실행할 수 없습니다.")


--- 모델 2: 유사 질문 생성 실행 예시 ---
원본 질문: 로보어드바이저가 가지고 있는 장점은 무엇이며, 특히 소액 자산가에게 어떤 이점을 제공하는지 설명하세요.
생성 목표 개수: 5

[LLM 생성 결과]
1. 로보어드바이저의 장점이 뭐고, 소액 투자자한테 어떤 이득이 있는지 알려줘.
2. 로보어드바이저가 가진 이점은 뭔가요? 특히 작은 자산을 가진 사람들에게 어떤 도움이 되나요?
3. 로보어드바이저의 좋은 점이 뭐야? 소액 자산가들한테는 어떤 혜택이 있을까?
4. 로보어드바이저가 제공하는 장점이 뭔지, 특히 자산이 적은 사람들한테 어떤 이점이 있는지 궁금해.
5. 로보어드바이저의 이점이 뭐야? 특히 소액 자산가에게 어떤 유리한 점이 있는지 설명해줘.


In [18]:
# --- 모델 3: 답변 스타일 변환 및 어조 자동 조정 모델 ---
def transform_answer_style(original_answer: str, target_style: str, llm: ChatOpenAI) -> str:
    """
    원본 답변의 내용을 유지하면서 지정된 스타일로 변환하는 함수.
    """
    prompt_text = """
    당신은 금융 설명을 다양한 스타일로 변환하는 AI 카피라이터입니다. 다음 [원본 답변]의 핵심 정보와 정확성은 그대로 유지하면서, [목표 스타일]에 맞게 문체를 변환하여 다시 작성해주세요.

    [원본 답변]
    {answer}

    [목표 스타일]
    {style}

    [스타일 변환된 답변]
    """
    prompt = PromptTemplate(input_variables=["answer", "style"], template=prompt_text)
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.invoke({"answer": original_answer, "style": target_style})
    return response['text']


In [19]:
# 모델 3 실행 예시
print("\n--- 모델 3: 답변 스타일 변환 실행 예시 ---")
if not df.empty and len(df) > 2:
    sample_index_style = 2
    sample_answer_for_style = df.loc[sample_index_style, 'answer_B'] # 품질 좋은 답변 B 사용
    target_style_friendly = "매우 친근하고 초등학생도 이해할 수 있게 쉬운 비유 사용"
    target_style_professional = "금융 전문가처럼 전문 용어를 사용하여 상세하고 깊이 있게"
    print(f"원본 답변 (일부): {sample_answer_for_style[:200]}...")

    print(f"\n[LLM 변환 결과 (스타일: {target_style_friendly})]")
    transformed_answer_friendly = transform_answer_style(sample_answer_for_style, target_style_friendly, llm)
    print(transformed_answer_friendly)

    print(f"\n[LLM 변환 결과 (스타일: {target_style_professional})]")
    transformed_answer_professional = transform_answer_style(sample_answer_for_style, target_style_professional, llm)
    print(transformed_answer_professional)
else:
    print("데이터프레임이 비어 있거나 샘플 수가 부족하여 실행할 수 없습니다.")


--- 모델 3: 답변 스타일 변환 실행 예시 ---
원본 답변 (일부): 로보어드바이저(Robo-Advisor)는 인공지능(AI) 및 알고리즘을 기반으로 투자 자산 관리 서비스를 제공하는 온라인 플랫폼입니다. 이 기술은 인간 투자 고문 없이도 사용자에게 맞춤형 투자 조언과 자산 배분을 제공하는 것을 목표로 합니다. 여기서는 로보어드바이저의 의미와 그 서비스가 제공되는 방식을 자세히 설명하겠습니다.

### 1. 로보어드바이저의 ...

[LLM 변환 결과 (스타일: 매우 친근하고 초등학생도 이해할 수 있게 쉬운 비유 사용)]
로보어드바이저는 마치 똑똑한 로봇 친구가 여러분의 저금통을 관리해주는 것과 같아요. 이 로봇 친구는 컴퓨터 머리를 가지고 있어서, 여러분이 돈을 어떻게 모으고 쓸지 똑똑하게 도와줄 수 있답니다. 이제 이 로봇 친구가 어떻게 일하는지 알아볼까요?

### 1. 로보어드바이저란?
로보어드바이저는 이런 특징을 가지고 있어요:
- **자동으로 돈 관리하기**: 여러분이 원하는 목표와 위험을 얼마나 감수할 수 있는지를 알려주면, 로봇 친구가 알아서 돈을 잘 나눠서 관리해줘요.
- **저렴한 비용**: 사람에게 돈을 맡기는 것보다 훨씬 적은 돈으로 이 로봇 친구의 도움을 받을 수 있어요.
- **누구나 사용 가능**: 돈이 많든 적든, 누구나 쉽게 사용할 수 있도록 만들어졌어요.

### 2. 로보어드바이저 서비스 사용 방법
로보어드바이저를 사용하는 방법은 다음과 같아요:

#### 1. 가입하고 질문에 답하기
먼저 로봇 친구에게 가입하고, 여러분의 나이, 목표(예: 장난감 사기, 여행 가기 등), 돈을 모으고 싶은 기간, 그리고 위험을 얼마나 감수할 수 있는지를 알려줘요. 이 정보는 로봇 친구가 여러분에게 딱 맞는 계획을 세우는 데 필요해요.

#### 2. 돈 나누기
여러분이 답한 내용을 바탕으로 로봇 친구가 돈을 어디에 투자할지 알려줘요. 주식, 채권, 부동산 같은 다양한 곳에 돈을 나눠서 투자해요. 이렇게 하면 위험을 줄이고 더 많은 돈을

In [20]:
# --- 모델 4: 콘텐츠 부족 영역 자동 발굴 모델 (VectorDB 연동 필요) ---
# VectorDB 검색 결과에서 낮은 유사도 점수를 보인 질문 목록을 가정
def identify_content_gaps(difficult_queries: list[str], llm: ChatOpenAI) -> str:
    """
    답변하기 어려웠던 질문 목록을 분석하여 콘텐츠가 부족한 주제 영역을 요약하는 함수.
    """
    prompt_text = """
    당신은 챗봇 운영 데이터를 분석하는 전문가입니다. 다음 [답변 어려운 질문 목록]은 현재 지식 베이스로는 답변하기 어려웠던 사용자 질문들입니다.
    이 질문들에서 공통적으로 나타나는 주요 금융 관련 주제나 키워드를 분석하고, 콘텐츠 보강이 필요한 영역 3가지를 요약하여 알려주세요.

    [답변 어려운 질문 목록]
    {query_list}

    [콘텐츠 부족 주제 영역 요약 (3가지)]
    1.
    2.
    3.
    """
    prompt = PromptTemplate(input_variables=["query_list"], template=prompt_text)
    query_list_str = "\n".join([f"- {q}" for q in difficult_queries])
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.invoke({"query_list": query_list_str})
    return response['text']

In [21]:
# 모델 4 실행 예시
print("\n--- 모델 4: 콘텐츠 부족 영역 발굴 실행 예시 ---")
sample_difficult_queries = [
    "최근 발표된 금리 인상 예외 조항에 대해 자세히 알려줘",
    "ISA 계좌 비과세 한도 초과 시 세금 처리 방법",
    "해외 부동산 투자 시 환헷지 전략과 수수료 비교",
    "특정 지역 재개발 조합원 자격 요건 변동 사항 문의",
    "금리 스왑 커브 역전 현상이 향후 경제에 미칠 영향 분석"
]
print(f"분석 대상 질문 목록: {sample_difficult_queries}")
print("\n[LLM 분석 결과 (콘텐츠 부족 추정 영역)]")
gap_analysis_result = identify_content_gaps(sample_difficult_queries, llm)
print(gap_analysis_result)


--- 모델 4: 콘텐츠 부족 영역 발굴 실행 예시 ---
분석 대상 질문 목록: ['최근 발표된 금리 인상 예외 조항에 대해 자세히 알려줘', 'ISA 계좌 비과세 한도 초과 시 세금 처리 방법', '해외 부동산 투자 시 환헷지 전략과 수수료 비교', '특정 지역 재개발 조합원 자격 요건 변동 사항 문의', '금리 스왑 커브 역전 현상이 향후 경제에 미칠 영향 분석']

[LLM 분석 결과 (콘텐츠 부족 추정 영역)]
[콘텐츠 부족 주제 영역 요약 (3가지)]

1. **금리 및 금융 정책 변화**: 최근 발표된 금리 인상 예외 조항이나 금리 스왑 커브 역전 현상과 같은 주제는 금융 정책의 변화와 그에 따른 경제적 영향을 이해하는 데 필요한 심층적인 정보가 부족합니다. 이러한 주제에 대한 콘텐츠를 보강하여 사용자가 최신 금융 정책과 그 영향에 대해 더 잘 이해할 수 있도록 해야 합니다.

2. **세금 및 계좌 관리**: ISA 계좌의 비과세 한도 초과 시 세금 처리 방법과 같은 주제는 개인 금융 관리와 관련된 세부적인 세금 규정에 대한 정보가 부족합니다. 사용자가 계좌 관리와 관련된 세금 문제를 명확히 이해할 수 있도록 관련 콘텐츠를 강화할 필요가 있습니다.

3. **해외 투자 및 환율 관리**: 해외 부동산 투자 시 환헷지 전략과 수수료 비교와 같은 주제는 국제 투자와 관련된 환율 관리 및 비용 구조에 대한 정보가 부족합니다. 사용자가 해외 투자 시 고려해야 할 환율 위험과 수수료에 대한 명확한 지침을 제공할 수 있도록 콘텐츠를 보강해야 합니다.


In [22]:
# --- 모델 5: Q&A 데이터 생성 및 검증 지원 모델 ---
def generate_qa_pair(topic: str, llm: ChatOpenAI, style_example: str = "") -> tuple[str, str]:
    """
    주어진 주제에 대해 자연스러운 질문과 고품질 답변 쌍 초안을 생성하는 함수.
    """
    style_guide = f"\n[답변 스타일 가이드]\n답변은 다음 설명을 참고하여 명확하고 상세하며 예시를 포함하여 작성합니다:\n{style_example}" if style_example else ""
    prompt_text_gen = """
    당신은 금융 교육 콘텐츠 작성 전문가입니다. 다음 [주제]에 대해 사용자가 자연스럽게 질문할 만한 내용을 예상하여 [질문]을 만들고, 그 질문에 대한 [답변] 초안을 작성해주세요.
    답변은 정확하고, 이해하기 쉬우며, 필요한 경우 간단한 예시를 포함해야 합니다.{style_guide}
    결과는 JSON 형식으로 "question"과 "answer" 키를 포함하여 반환해주세요.

    [주제]
    {topic}

    [생성된 Q&A (JSON 형식)]
    """
    prompt_gen = PromptTemplate(input_variables=["topic", "style_guide"], template=prompt_text_gen)
    chain_gen = LLMChain(llm=llm, prompt=prompt_gen)
    try:
        response_gen = chain_gen.invoke({"topic": topic, "style_guide": style_guide})
        json_response_str = response_gen['text'].strip()
        if json_response_str.startswith("```json"): json_response_str = json_response_str[7:]
        if json_response_str.endswith("```"): json_response_str = json_response_str[:-3]
        qa_pair = json.loads(json_response_str.strip())
        return qa_pair.get("question", ""), qa_pair.get("answer", "")
    except Exception as e:
        print(f"Q&A 생성 또는 JSON 파싱 중 오류 발생: {e}")
        print(f"LLM 응답 원본: {response_gen['text'] if 'response_gen' in locals() else 'N/A'}")
        return "", ""

def validate_qa_pair(question: str, answer: str, llm: ChatOpenAI, source_text: str = None) -> str:
    """
    생성된 Q&A 쌍의 정확성, 명확성 등을 검토하고 피드백을 제공하는 함수.
    """
    source_section = f"\n[참고 원문]\n{source_text}" if source_text else ""
    prompt_text_val = """
    당신은 금융 콘텐츠 감수 전문가입니다. 다음 [생성된 Q&A]가 사실 관계에 부합하는지, 설명이 명확한지, 오해의 소지는 없는지 검토해주세요.{source_section}
    만약 문제점이 있다면 구체적으로 지적하고, 개선 방안을 제안해주세요. 문제가 없다면 "검토 결과 특이사항 없음"이라고 답변해주세요.

    [생성된 Q&A]
    질문: {question}
    답변: {answer}

    [검토 결과 및 피드백]
    """
    prompt_val = PromptTemplate(input_variables=["question", "answer", "source_section"], template=prompt_text_val)
    chain_val = LLMChain(llm=llm, prompt=prompt_val)
    response_val = chain_val.invoke({"question": question, "answer": answer, "source_section": source_section})
    return response_val['text']

In [23]:
# 모델 5 실행 예시
print("\n--- 모델 5: Q&A 생성 및 검증 지원 실행 예시 ---")
if not df.empty:
    topic_to_generate = "주식 배당금 계산 방법"
    style_guideline = df['preference_desc'].sample(1).iloc[0] if 'preference_desc' in df.columns and not df.empty else ""
    print(f"생성 주제: {topic_to_generate}")

    # 1. Q&A 쌍 생성
    generated_q, generated_a = generate_qa_pair(topic_to_generate, llm, style_guideline)

    if generated_q and generated_a:
        print("\n[LLM 생성 Q&A 초안]")
        print(f"생성된 질문: {generated_q}")
        print(f"생성된 답변: {generated_a}")

        # 2. 생성된 Q&A 검증 (source_text 없이 자체 논리 검토 요청)
        print("\n[LLM 검증 결과]")
        validation_feedback = validate_qa_pair(generated_q, generated_a, llm)
        print(validation_feedback)
    else:
        print("Q&A 쌍 생성에 실패했습니다.")
else:
    print("데이터프레임이 비어 있어 실행할 수 없습니다.")


--- 모델 5: Q&A 생성 및 검증 지원 실행 예시 ---
생성 주제: 주식 배당금 계산 방법

[LLM 생성 Q&A 초안]
생성된 질문: 주식 배당금은 어떻게 계산하나요?
생성된 답변: 주식 배당금은 주식 보유자가 기업의 이익 일부를 배당금 형태로 받는 것을 의미합니다. 배당금 계산은 일반적으로 주당 배당금(DPS)과 보유 주식 수를 곱하여 이루어집니다. 예를 들어, 특정 기업이 주당 2달러의 배당금을 지급한다고 가정하고, 당신이 그 기업의 주식을 100주 보유하고 있다면, 총 배당금은 2달러 * 100주 = 200달러가 됩니다. 

배당금은 기업의 이사회에서 결정하며, 배당금 지급 여부와 금액은 기업의 수익성, 현금 흐름, 미래 투자 계획 등 다양한 요인에 따라 달라질 수 있습니다. 또한, 배당금 지급일과 기준일을 확인하는 것도 중요합니다. 기준일은 배당금을 받을 수 있는 주주를 확정하는 날짜이며, 지급일은 실제로 배당금이 지급되는 날입니다. 

이 외에도 배당 수익률이라는 개념이 있는데, 이는 주당 배당금을 주가로 나눈 값으로, 투자자가 주식에서 얻을 수 있는 수익률을 나타냅니다. 예를 들어, 주가가 50달러이고 주당 배당금이 2달러라면, 배당 수익률은 2달러 / 50달러 = 4%가 됩니다. 이러한 계산을 통해 투자자는 배당금이 자신의 투자 전략에 얼마나 부합하는지 평가할 수 있습니다.

[LLM 검증 결과]
검토 결과 특이사항 없음. 

설명은 주식 배당금의 계산 방법과 관련된 기본적인 개념을 명확하게 전달하고 있으며, 배당금 결정 과정과 관련된 추가적인 정보도 잘 설명되어 있습니다. 배당 수익률에 대한 설명도 적절하며, 투자자가 배당금을 평가하는 데 유용한 정보를 제공합니다. 전반적으로 오해의 소지가 없고, 사실 관계에 부합합니다.


In [24]:
import gradio as gr

# 여기서는 모델 1 (품질 평가)을 데모로 만든다고 가정
def gradio_interface(question, answer):
    if not df.empty:
        # 실제로는 df에서 criteria_examples를 가져오는 로직 추가 가능
        criteria = df['preference_desc'].sample(1).iloc[0] if 'preference_desc' in df.columns and not df.empty else ""
        return evaluate_answer_quality(question, answer, llm, criteria)
    else:
        return "데이터 로딩 실패로 평가 불가"

iface = gr.Interface(fn=gradio_interface,
                     inputs=[gr.Textbox(label="질문", placeholder="평가할 질문 입력..."),
                             gr.Textbox(label="답변", placeholder="평가할 답변 입력...")],
                     outputs=gr.Textbox(label="평가 및 개선 제안 결과"),
                     title="챗봇 답변 품질 평가 도우미",
                     description="질문과 답변을 입력하면 품질을 평가하고 개선점을 제안합니다.")

if __name__ == "__main__": # Colab에서는 직접 실행보다는 셀 실행이 일반적
    # iface.launch() # 데모 실행
    pass # Colab에서는 마지막 셀에서 실행하는 것이 더 적합할 수 있습니다.